In [1]:
import pyspark.sql.types as T
import pyspark.sql.functions as F

from pyspark import SparkContext, SparkConf, SQLContext

from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import pandas_udf
from pyspark.sql.window import Window

import pandas as pd
import numpy as np

from datetime import date, datetime, timedelta, timezone
import os
import json
import requests
import time
import yaml
from itertools import chain

import concurrent.futures
from threading import Lock

In [2]:
appName = "PySpark TFT puuids"
master = "local[*]"
conf = SparkConf() \
    .setAppName(appName) \
    .setMaster(master) \
    .set("spark.executor.memory", "40g") \
    .set("spark.driver.memory", "40g") \
    .set("spark.executor.memoryOverhead", "8g") \
    .set("spark.local.dir", "/home/mai/spark-temp") \
    .set("spark.sql.session.timeZone", "UTC") \
    .set("spark.dynamicAllocation.enabled", "true") \
    .set("spark.dynamicAllocation.minExecutors", "2") \
    .set("spark.dynamicAllocation.maxExecutors", "50") \
    .set("spark.speculation", "true") 
   
sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

25/02/22 14:13:59 WARN Utils: Your hostname, LAPTOP-4O0SI9BK resolves to a loopback address: 127.0.1.1; using 172.30.70.20 instead (on interface eth0)
25/02/22 14:13:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/22 14:14:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/22 14:14:01 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 39218)
Traceba

In [3]:
df = spark.read.parquet('./data/tft_match_data_cleaned/matches_euw1_CHALLENGER_I.parquet')

In [4]:
df.printSchema()

root
 |-- metadata_data_version: string (nullable = true)
 |-- metadata_match_id: string (nullable = true)
 |-- info_endOfGameResult: string (nullable = true)
 |-- info_gameCreation: long (nullable = true)
 |-- info_game_datetime: long (nullable = true)
 |-- info_game_length: double (nullable = true)
 |-- info_game_version: string (nullable = true)
 |-- info_mapId: long (nullable = true)
 |-- info_queueId: long (nullable = true)
 |-- info_tft_game_type: string (nullable = true)
 |-- info_tft_set_core_name: string (nullable = true)
 |-- info_tft_set_number: long (nullable = true)
 |-- gold_left: long (nullable = true)
 |-- last_round: long (nullable = true)
 |-- level: long (nullable = true)
 |-- partner_group_id: integer (nullable = true)
 |-- placement: long (nullable = true)
 |-- players_eliminated: long (nullable = true)
 |-- puuid: string (nullable = true)
 |-- riotIdGameName: string (nullable = true)
 |-- riotIdTagline: string (nullable = true)
 |-- skill_tree: integer (nullable =

In [24]:
# get all unique unit names
units = df.select(F.explode('units').alias('units')) \
          .select(F.col('units.character_id'))\
          .distinct()\
          .collect()

In [25]:
units = [i['character_id'].split('_')[-1] for i in units]

units

['Gremlin',
 'NunuWillump',
 'JayceSummon',
 'Illaoi',
 'Nami',
 'Sett',
 'Corki',
 'Twitch',
 'DrMundo',
 'Viktor',
 'TwistedFate',
 'Trundle',
 'Cassiopeia',
 'Rumble',
 'Lieutenant',
 'Zoe',
 'Garen',
 'Morgana',
 'Vex',
 'Elise',
 'Camille',
 'Ambessa',
 'Tristana',
 'Jinx',
 'Vladimir',
 'Draven',
 'Malzahar',
 'Nocturne',
 'Gangplank',
 'Chainsaw',
 'Ziggs',
 'Leona',
 'Fish',
 'Lux',
 'FlyGuy',
 'Darius',
 'RenataGlasc',
 'Blitzcrank',
 'Zeri',
 'swain',
 'Akali',
 'Caitlyn',
 'Sion',
 'KogMaw',
 'Jayce',
 'Urgot',
 'Mordekaiser',
 'Amumu',
 'Red',
 'Rell',
 'Ekko',
 'LeBlanc',
 'MissMage',
 'Silco',
 'elise',
 'Ezreal',
 'Warwick',
 'Heimerdinger',
 'Prime',
 'Vi',
 'Shooter',
 'Singed',
 'Blue',
 'Zyra',
 'jinx',
 'Irelia',
 'Beardy']

In [27]:
len(units)

67

In [5]:
df.withColumn('num_units', F.size(F.col('units')))\
    .withColumn('num_traits', F.size(F.col('traits')))\
    .withColumn('num_units_tier_3', F.size(F.filter(F.col('units'), lambda x: x.tier == 3)))\
    .withColumn('highest_unit_rarity', F.array_max(F.col('units.rarity')))\
    .withColumn('units_with_highest_unit_rarity', F.filter(F.col('units'), lambda x: x.rarity == F.col('highest_unit_rarity')))\
    .withColumn('highest_tier_of_highest_unit_rarity', F.array_max(F.transform('units_with_highest_unit_rarity', lambda x: x.tier)))\
    .withColumn('num_items', F.aggregate("units", F.lit(0), lambda acc, x: acc + F.size(x.itemNames)))\
    .select('metadata_match_id', 'puuid', 'placement', 'num_units', 'num_traits', 'num_items',
            'highest_unit_rarity', 'highest_tier_of_highest_unit_rarity', 'num_units_tier_3', 'level')\
    .orderBy('metadata_match_id', 'placement')\
    .show(8)

+-----------------+--------------------+---------+---------+----------+---------+-------------------+-----------------------------------+----------------+-----+
|metadata_match_id|               puuid|placement|num_units|num_traits|num_items|highest_unit_rarity|highest_tier_of_highest_unit_rarity|num_units_tier_3|level|
+-----------------+--------------------+---------+---------+----------+---------+-------------------+-----------------------------------+----------------+-----+
|  EUW1_7277890033|xmkFl1BkSaRjzuz3m...|        1|       11|        12|       16|                  9|                                  1|               0|   10|
|  EUW1_7277890033|o8slPR9SLMExDt91J...|        2|        9|        13|       14|                  8|                                  1|               1|    9|
|  EUW1_7277890033|Yxuy3gv-3zlkLdQHJ...|        3|        8|         9|       13|                  6|                                  1|               2|    8|
|  EUW1_7277890033|J0lyE3mOPaXKvF5

In [6]:
df.withColumn('num_units', F.size(F.col('units')))\
    .withColumn('num_traits', F.size(F.col('traits')))\
    .withColumn('num_units_tier_3', F.size(F.filter(F.col('units'), lambda x: x.tier == 3)))\
    .withColumn('highest_unit_rarity', F.array_max(F.col('units.rarity')))\
    .withColumn('num_items', F.aggregate("units", F.lit(0), lambda acc, x: acc + F.size(x.itemNames)))\
    .select('metadata_match_id', 'puuid', 'placement', 'num_units', 'num_traits', 'num_items',
            'highest_unit_rarity', 'num_units_tier_3', 'level')\
    .orderBy('metadata_match_id', 'placement')\
    .show(8)

+-----------------+--------------------+---------+---------+----------+---------+-------------------+----------------+-----+
|metadata_match_id|               puuid|placement|num_units|num_traits|num_items|highest_unit_rarity|num_units_tier_3|level|
+-----------------+--------------------+---------+---------+----------+---------+-------------------+----------------+-----+
|  EUW1_7277890033|xmkFl1BkSaRjzuz3m...|        1|       11|        12|       16|                  9|               0|   10|
|  EUW1_7277890033|o8slPR9SLMExDt91J...|        2|        9|        13|       14|                  8|               1|    9|
|  EUW1_7277890033|Yxuy3gv-3zlkLdQHJ...|        3|        8|         9|       13|                  6|               2|    8|
|  EUW1_7277890033|J0lyE3mOPaXKvF55h...|        4|        8|         7|       12|                  8|               5|    8|
|  EUW1_7277890033|9FCno9_0fI0PAIYT7...|        5|        9|        10|       16|                  9|               0|    8|


In [7]:
df.where(F.col('metadata_match_id') == 'EUW1_7291286802')\
    .select('puuid', 'gold_left', 'placement', 'last_round', 'level', 'players_eliminated', 'total_damage_to_players', F.explode('traits').alias('traits'), 'units')\
    .select('puuid', 'gold_left', 'placement', 'last_round', 'level', 'players_eliminated', 'total_damage_to_players', 'traits.name', 'traits.tier_current',
           'traits.style', 'traits.tier_total').show(truncate=False)

+------------------------------------------------------------------------------+---------+---------+----------+-----+------------------+-----------------------+-----------------+------------+-----+----------+
|puuid                                                                         |gold_left|placement|last_round|level|players_eliminated|total_damage_to_players|name             |tier_current|style|tier_total|
+------------------------------------------------------------------------------+---------+---------+----------+-----+------------------+-----------------------+-----------------+------------+-----+----------+
|4XZMPNXlRKTe9jXDeUzFFQoGOPNIOrjZfIY7vcECXNiXk1xKJgcz3yN5goB3oa4xoFLD8mTj420okQ|44       |5        |30        |8    |0                 |91                     |TFT13_Academy    |2           |2    |4         |
|4XZMPNXlRKTe9jXDeUzFFQoGOPNIOrjZfIY7vcECXNiXk1xKJgcz3yN5goB3oa4xoFLD8mTj420okQ|44       |5        |30        |8    |0                 |91                     |TFT1

In [22]:
df.where(F.col('metadata_match_id') == 'EUW1_7291286802')\
    .select('puuid', 'gold_left', 'placement', 'last_round', 'level', 'players_eliminated', 'total_damage_to_players', F.explode('units').alias('units'))\
    .select('puuid', 'gold_left', 'placement', 'last_round', 'level', 'players_eliminated', 'total_damage_to_players', 'units.character_id', 'units.itemNames',
           'units.rarity', 'units.tier').show(100)

+--------------------+---------+---------+----------+-----+------------------+-----------------------+------------------+--------------------+------+----+
|               puuid|gold_left|placement|last_round|level|players_eliminated|total_damage_to_players|      character_id|           itemNames|rarity|tier|
+--------------------+---------+---------+----------+-----+------------------+-----------------------+------------------+--------------------+------+----+
|4XZMPNXlRKTe9jXDe...|       44|        5|        30|    8|                 0|                     91|      TFT13_Irelia|                  []|     0|   2|
|4XZMPNXlRKTe9jXDe...|       44|        5|        30|    8|                 0|                     91|       TFT13_Leona|                  []|     1|   2|
|4XZMPNXlRKTe9jXDe...|       44|        5|        30|    8|                 0|                     91|        TFT13_Rell|[TFT_Item_Bramble...|     1|   1|
|4XZMPNXlRKTe9jXDe...|       44|        5|        30|    8|           